<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-данных" data-toc-modified-id="Импорт-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт данных</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Функции-преобразования-слов-в-векторы-признаков" data-toc-modified-id="Функции-преобразования-слов-в-векторы-признаков-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Функции преобразования слов в векторы признаков</a></span></li><li><span><a href="#Создание-словорей-для-текстов" data-toc-modified-id="Создание-словорей-для-текстов-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Создание словорей для текстов</a></span></li><li><span><a href="#Переменные-словарей" data-toc-modified-id="Переменные-словарей-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Переменные словарей</a></span></li><li><span><a href="#Очистка-и-преобразование-тестовых-полей" data-toc-modified-id="Очистка-и-преобразование-тестовых-полей-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Очистка и преобразование тестовых полей</a></span></li><li><span><a href="#Объявление-данных-для-обучения" data-toc-modified-id="Объявление-данных-для-обучения-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Объявление данных для обучения</a></span></li><li><span><a href="#Создание-модели-для-обучения" data-toc-modified-id="Создание-модели-для-обучения-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Создание модели для обучения</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#КНН" data-toc-modified-id="КНН-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>КНН</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Предсказание" data-toc-modified-id="Предсказание-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Предсказание</a></span></li><li><span><a href="#Подгоотвка-выходных-данных" data-toc-modified-id="Подгоотвка-выходных-данных-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Подгоотвка выходных данных</a></span></li><li><span><a href="#Сохранение-результата" data-toc-modified-id="Сохранение-результата-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Сохранение результата</a></span></li></ul></div>

# Обучение

## Импорт данных

In [1]:
import pandas as pd
import scipy.sparse as sp
from sklearn.svm import LinearSVC
from tqdm import tqdm
import time

## Загрузка данных

In [ ]:
%%time
df_isx = pd.read_excel('VR-ЛМ.xlsx')
# df_isx = df_isx[df_isx.apply(lambda row: True if  row['PRED_INSTR'][:6]== 'ЛМ7002' else False, axis=1)== True]
df = df_isx
df2 = df.loc[:,['PRED_INSTR','NAIM_1','CEH','NAIM_2','MASSA']]
df2['PRED'] = df2.loc[:,['PRED_INSTR']].apply(lambda x: x, axis=1)
df2['NAIM'] = df2.loc[:,['NAIM_1']].apply(lambda x: x, axis=1)
df = pd.concat([df,df2],sort=False)
df['MASSA'] = df['MASSA'].fillna(0)
df['MASSA'] = df.loc[:,['PRED','NAIM_2']].apply(lambda x: int(df.loc[:,['PRED','NAIM_2','MASSA']][df['PRED'] == x[0]].MASSA.median()), axis=1)
df['NAIM_2'] = df.loc[:,['PRED','NAIM_2']].apply(lambda x: str(df.loc[:,['PRED','NAIM_2']][df['PRED'] == x[0]].NAIM_2.to_list()), axis=1)
df = df.loc[:,['PRED', 'NAIM', 'PRED_INSTR', 'NAIM_1', 'NAIM_2', 'MASSA', 'TIP_INSTR']].drop_duplicates()
df.head()

In [ ]:
df.shape

## Функции преобразования слов в векторы признаков

In [7]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re

In [8]:
def klean_text(text):
#     Приведение к нижнему регистру 
    text = text.apply(lambda X: str(X).lower())
#     Замена ё на е
    text = text.apply(lambda X: re.sub(r'ё','е', str(X)))
#     Замена ё на е    
    text = text.apply(lambda X: re.sub(r'[:;=.,()|\d|-]', ' ', str(X)))
    return text

def nan_renam(text):
    text = np.array(text)
    for n in range(0, len(text)):
        if text[n] == 'nan':
            text[n] = ''
    return text

# Соединяет список в один

def list_combining(text):
    text1 =[]
    for t in text:      
        if isinstance(t, list):
            t = ' '.join(t)
            text1.append(t)
    print(len(text1))
    if len(text1)>0:
        text = text1
    return text

def cv_text_transform(text):
    text = list_combining(text)
    count = CountVectorizer()
    docs = nan_renam(text)
    bag = count.fit_transform(docs)
    cv = CountVectorizer()
    cv.fit(docs)
#     print(cv.get_feature_names())
    return cv

def cv_text_transform_tiporazmer(text, tip_list):
    tip_dikt = {}
    tip_dikt1 = {}
    tip_dikt2 = {}

    count = CountVectorizer()
    count.fit_transform(text)
    
    for tip in tip_list:
        if tip in count.vocabulary_:
            tip_dikt[tip] = count.vocabulary_[tip]
    
    for i in range(len([*tip_dikt])):
        word = [*tip_dikt][i]
        obr = 0
        for nam in range(len(word)):
            if nam>0 and word[nam].isdigit() == False:
                obr = int(word[1:nam])
                break

        if obr == 0: obr = int(word[1:])
    #     print ([obr,[*tip_dikt][i]])
        if obr not in tip_dikt1:
            tip_dikt1[obr] = tip_dikt[[*tip_dikt][i]]
    #     print ([[*tip_dikt][i],obr, tip_dikt1[obr]])
        tip_dikt2[[*tip_dikt][i]] = tip_dikt1[obr]
    count.vocabulary_ = tip_dikt2
    return count



# Преобрает данные с использованием словарей
def converter (df, cv_NAIM_train, cv_NAIM_dse_train,tip_list):
#     tiporazmer_train = tiporazmer(cv_NAIM_train.transform(df.NAIM.tolist()).toarray(),tip_list, cv_NAIM_train)
    NAIM_train = cv_NAIM_train.transform(df.NAIM.tolist()).toarray()
    NAIM_dse_train = cv_NAIM_dse_train.transform(list_combining(df.NAIM_2.tolist()))
    Tiporazmer = cv_NAIM_train_tiporazm.transform(df.NAIM.tolist()).toarray()
    MASSA = np.array(df.MASSA,dtype = int, ndmin = 2).transpose()
    print([NAIM_train.shape, NAIM_dse_train.shape, Tiporazmer.shape])
    X = sp.hstack([NAIM_train, NAIM_dse_train,Tiporazmer])
#     X = NAIM_train
#     X = sp.hstack([NAIM_train, NAIM_dse_train, MASSA])
    return X

## Создание словорей для текстов

In [9]:
# Слова из задания
cv_NAIM_train = cv_text_transform(df.NAIM.tolist())
# Слова из целевой детали
cv_NAIM_dse_train = cv_text_transform(df.NAIM_2.tolist())

0
0


## Переменные словарей

In [10]:
TIP_INSTR_dikt = {r'R':0, r'СТО':1}
grop_dikt = {}
mail_dikt ={}
pered_dikt={}
tip_list = [i for i in cv_NAIM_train.get_feature_names() if i[0] in ['м', 'ф', 'r'] and i[1].isdigit()]

## Очистка и преобразование тестовых полей

In [11]:
# Типоразмерная харрактеристика
cv_NAIM_train_tiporazm = cv_text_transform_tiporazmer(df.NAIM.tolist(), tip_list)

In [12]:
# Убрать типоразмерные харрактеристики
for kei in [*cv_NAIM_train_tiporazm.vocabulary_]:
    del cv_NAIM_train.vocabulary_[kei]

In [13]:
len(cv_NAIM_train.vocabulary_)
cv_NAIM_train.vocabulary_

{'приспособление': 210,
 'для': 97,
 'обработки': 165,
 'отв': 171,
 'секторах': 243,
 'кольца': 119,
 'шаблон': 312,
 'на': 157,
 'профиль': 213,
 'схема': 264,
 'установки': 283,
 'цвд': 305,
 'ст': 253,
 'форрест': 297,
 'балансировки': 62,
 'поршня': 197,
 'сверло': 239,
 '23': 18,
 'удл': 279,
 'шайбы': 314,
 'подкладные': 189,
 'под': 188,
 'колпачковые': 118,
 'гайки': 80,
 'лм7019': 141,
 '0535': 0,
 'линейка': 136,
 'поверочная': 186,
 'паз': 174,
 '120н9': 6,
 'лм7856': 144,
 '0677': 2,
 'кольцо': 120,
 'калибр': 108,
 'разметки': 221,
 'отверстий': 172,
 'пазов': 175,
 'форсунки': 298,
 'строповки': 260,
 'кантовки': 111,
 'канавку': 109,
 '30': 27,
 'град': 90,
 'эл': 322,
 'гидроиспытания': 83,
 'совмесно': 249,
 'крышками': 131,
 'черт': 311,
 '2338135сб': 20,
 '2338145сб': 21,
 'транспортировки': 272,
 'фланца': 295,
 'трубы': 274,
 'спец': 251,
 'скоба': 246,
 'размер': 220,
 '18а11': 13,
 '29': 25,
 'упор': 280,
 'от': 170,
 'выпадания': 79,
 'цапфы': 304,
 'из': 105,


In [14]:
# # Отобразить таблицу типоразмерных харрактерристик
# matr = NAIM_train
# df_dikt = pd.DataFrame(matr,columns = cv_NAIM_train.get_feature_names(), index = df.NAIM.tolist())
# df_dikt[tip_list]

In [15]:
# Масштабирование
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

## Объявление данных для обучения

In [16]:
X = converter(df, cv_NAIM_train, cv_NAIM_dse_train,tip_list)
y = df.PRED_INSTR
[X.shape, y.shape]

0
[(150, 301), (150, 391), (150, 23)]


[(150, 715), (150,)]

## Создание модели для обучения

In [17]:
from sklearn.model_selection import train_test_split, StratifiedKFold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                          random_state=17)
[X_train.shape, y_train.shape]

[(135, 715), (135,)]

## Логистическая регрессия

In [1]:
%%time
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(random_state=17, solver='lbfgs',multi_class='multinomial')
logit.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
from sklearn import model_selection
model_selection.cross_val_score(logit, X_train, y_train)[0]

In [26]:
%%time
from sklearn.metrics import accuracy_score
tree_pred = logit.predict(X_test)
accuracy_score(y_test, tree_pred)

Wall time: 1.97 ms


0.7333333333333333

## КНН

In [27]:
%%time
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=2, n_jobs = 1)
classifier.fit(X_train, y_train)

Wall time: 999 µs


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [28]:
%%time
from sklearn.metrics import accuracy_score
test_predikt = classifier.predict(X_test)
accuracy_score(y_test, test_predikt)

Wall time: 4.98 ms


0.5333333333333333

In [29]:
from sklearn import model_selection
model_selection.cross_val_score(classifier, X_train, y_train)[0]

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


0.6086956521739131

# Предсказание

In [30]:
import random
l = len(df)
n = 140
i1 = random.randint(0, l-n-1)

In [31]:
df_out = df_isx.sample(frac= 0.1,random_state=i1)
df_out = df_out.loc[:,['PRED','NAIM','PRED_INSTR', 'NAIM_1', 'CEH', 'NAIM_2','MASSA']]
df_out.head()

,PRED,NAIM,PRED_INSTR,NAIM_1,CEH,NAIM_2,MASSA
164558,VR1890237,Схема строповки для кантовки и транспортирован...,ЛМ0868-6332,Схема строповки для кантовки и транспортировки...,133,ПЛИТА ВЫВОДОВ,1396.0
76587,VR1799852,Схема кантовки колена в процессе сборки-сварки,ЛМ0868-6334,Схема строповки для транспортировки и кантовки...,105,Патрубок,1610.0
189438,VR1718941,Схема строповки для кантовки и транспортировки,ЛМ0868-6333,Схема строповки для кантовки и транспортировки...,213,Заготовка обода диафрагмы 4ст. для ч.1479944,600.0
78275,VR1832473,Схема кантовки и транспортировки,ЛМ0868-6332,Схема строповки для кантовки и транспортировки...,133,СТЕНКА,2261.0
159574,VR1718742,Схема строповки для кантовки и транспортировки,ЛМ0868-6333,Схема строповки для кантовки и транспортировки...,133,Фланец,300.0


In [32]:
# Кодирование выборки
X_work = converter(df_out, cv_NAIM_train, cv_NAIM_dse_train,tip_list)

0
[(23759, 301), (23759, 391), (23759, 23)]


In [34]:
# Предсказание
work_predikt = logit.predict(X_work)
print(work_predikt.shape)

(23759,)


In [42]:
LM_predict_proba = []
LM_vern_otv = []
logit.predict_proba(X_work)
proba = logit.predict_proba(X_work)
for rang in range(proba.shape[1]):
    LM_predict_proba.append(max(proba[rang]))
    LM_vern_otv.append(proba[rang][df_out.PRED_INSTR.to_list().
                                   index(df_out.PRED_INSTR.to_list()[rang])])

In [43]:
naim_inst_dikt = pd.Series(df.NAIM_1.values,index=df.PRED_INSTR).to_dict()

In [45]:
df_out.shape

(23759, 8)

In [44]:
df_out['PRED_INSTR_PREDIKT'] = work_predikt
df_out['LM_predict_proba'] = LM_predict_proba
df_out['LM_vern_otv'] = LM_vern_otv
df_out['NAIM_INSTR_PREDIKT'] = df_out.loc[:,'PRED_INSTR_PREDIKT'].apply(lambda pred: naim_inst_dikt[pred])
df_out['False'] = df_out.loc[:,['PRED_INSTR','PRED_INSTR_PREDIKT']].apply(lambda x: x[0]==x[1], axis=1)
df_out[(df_out['False'] == False)]

ValueError: Length of values does not match length of index

In [ ]:
1 - df_out[(df_out['False'] == False)].shape[0] / df_out.shape[0]

## Подготовка данных

## Предсказание

## Подгоотвка выходных данных

## Сохранение результата

In [ ]:
df_out.to_excel('Поиск аналогов predict.xlsx', index = False)